#catboost baseline submission

public 기준 	0.6955106622

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.0 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import random
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import f1_score
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import catboost

# import eli5
# from eli5.sklearn import PermutationImportance

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv('./drive/MyDrive/open/train.csv')
test_df = pd.read_csv('./drive/MyDrive/open/test.csv')

train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

A = pd.DataFrame(train_x.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_x.shape[0]].index)
train_x.drop(columns=a,inplace=True)
test_x.drop(columns=a,inplace=True)

train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

Done.


In [ ]:
ttrain_y = train_x['Y_Class']
train_y = train_x['Y_Quality']
train_x = train_x.drop(columns=['Y_Class','Y_Quality'])

In [ ]:
clf = catboost.CatBoostRegressor(learning_rate=0.05, iterations=500, verbose=0)

In [ ]:
clf.fit(train_x, train_y, early_stopping_rounds=100, cat_features=['PRODUCT_CODE', 'LINE'])

In [ ]:
pre_preds = clf.predict(test_x)
pre_preds

In [ ]:
a = train_df[['Y_Class','Y_Quality']].groupby('Y_Class').agg(['mean', 'min', 'max', 'count'])
a

Y_Quality                          
             mean       min       max count
Y_Class                                    
0        0.520837  0.500856  0.525067    88
1        0.530253  0.525086  0.534843   407
2        0.542031  0.534951  0.578841   103

In [ ]:
preds=[]
for p in pre_preds:
  if p<=a[('Y_Quality','max')][0]:
    preds.append(0)
  elif p<=a[('Y_Quality','min')][2]:
    preds.append(1)
  else:
    preds.append(2)

In [ ]:
submit = pd.read_csv('./drive/MyDrive/open/sample_submission.csv')
submit['Y_Class'] = preds
submit.to_csv('./baseline_submission.csv', index=False)

#catboost baseline validation

0.5977182539682541

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv('./drive/MyDrive/open/train.csv')
test_df = pd.read_csv('./drive/MyDrive/open/test.csv')

train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

#쓰레기 변수 지우기
A = pd.DataFrame(train_x.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_x.shape[0]].index)
train_x.drop(columns=a,inplace=True)

In [ ]:
x_train, x_val= train_test_split(train_x, test_size=0.2, shuffle=True, random_state=37)

#결측값 처리 코드
x_train = x_train.fillna(0)
x_val = x_val.fillna(0)

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(x_train[i])
    x_train[i] = le.transform(x_train[i])

    for label in np.unique(x_val[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_val[i] = le.transform(x_val[i])
print('Done.')

Done.


In [ ]:
yy_train = x_train['Y_Class']
y_train = x_train['Y_Quality']
x_train = x_train.drop(columns=['Y_Quality','Y_Class'])

yy_val = x_val['Y_Class']
y_val = x_val['Y_Quality']
x_val = x_val.drop(columns=['Y_Quality','Y_Class'])

In [ ]:
clf = catboost.CatBoostRegressor(learning_rate=0.05, iterations=500, verbose=0)
clf.fit(x_train, y_train, early_stopping_rounds=100, cat_features=['PRODUCT_CODE', 'LINE'])

In [ ]:
from sklearn.metrics import f1_score
pre_preds = clf.predict(x_val)
print(clf.score(x_train,y_train))
print(clf.score(x_val,y_val))

0.9879335660550852
0.4635127370010562


In [ ]:
a = train_df[['Y_Class','Y_Quality']].groupby('Y_Class').agg(['mean', 'min', 'max', 'count'])
a

Y_Quality                          
             mean       min       max count
Y_Class                                    
0        0.520837  0.500856  0.525067    88
1        0.530253  0.525086  0.534843   407
2        0.542031  0.534951  0.578841   103

In [ ]:
preds=[]
for p in pre_preds:
  if p<=a[('Y_Quality','max')][0]:
    preds.append(0)
  elif p<=a[('Y_Quality','min')][2]:
    preds.append(1)
  else:
    preds.append(2)

In [ ]:
print(f1_score(yy_val, preds, average = 'macro'))

0.5977182539682541


결측치 처리 방법

1. 제거 (너무 많음 안됌)

2. 대치 (최빈값, 중앙값, 평균, similar case imputation, generalized imputation)

3. 예측 모델 (너무 많아서 오히려 에러 누적 가능성 높음)

# catboost baseline + 최빈값 결측치 처리
0.6273310023310024

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv('./drive/MyDrive/open/train.csv')
test_df = pd.read_csv('./drive/MyDrive/open/test.csv')

train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

A = pd.DataFrame(train_x.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_x.shape[0]].index)
train_x.drop(columns=a,inplace=True)

x_train, x_val= train_test_split(train_x, test_size=0.2, shuffle=True, random_state=37)

In [ ]:
mode_list=[]
for c in x_train.columns[4:]:
  mode_list.append(x_train[c].median())

In [ ]:
for i,c in enumerate(x_train.columns[4:]):
  x_train[c].fillna(mode_list[i],inplace=True)
  x_val[c].fillna(mode_list[i],inplace=True)

In [ ]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(x_train[i])
    x_train[i] = le.transform(x_train[i])

    for label in np.unique(x_val[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_val[i] = le.transform(x_val[i])
print('Done.')

Done.


In [ ]:
yy_train = x_train['Y_Class']
y_train = x_train['Y_Quality']
x_train = x_train.drop(columns=['Y_Quality','Y_Class'])

yy_val = x_val['Y_Class']
y_val = x_val['Y_Quality']
x_val = x_val.drop(columns=['Y_Quality','Y_Class'])

In [ ]:
clf = catboost.CatBoostRegressor(learning_rate=0.05, iterations=500, verbose=0)
clf.fit(x_train, y_train, early_stopping_rounds=100, cat_features=['PRODUCT_CODE', 'LINE'])

In [ ]:
from sklearn.metrics import f1_score
pre_preds = clf.predict(x_val)
print(clf.score(x_train,y_train))
print(clf.score(x_val,y_val))

0.9892685610253007
0.46758400914764564


In [ ]:
a = train_df[['Y_Class','Y_Quality']].groupby('Y_Class').agg(['mean', 'min', 'max', 'count'])
a

Y_Quality                          
             mean       min       max count
Y_Class                                    
0        0.520837  0.500856  0.525067    88
1        0.530253  0.525086  0.534843   407
2        0.542031  0.534951  0.578841   103

In [ ]:
preds=[]
for p in pre_preds:
  if p<=a[('Y_Quality','max')][0]:
    preds.append(0)
  elif p<=a[('Y_Quality','min')][2]:
    preds.append(1)
  else:
    preds.append(2)

In [ ]:
print(f1_score(yy_val, preds, average = 'macro'))

0.6379806198265391


# catboost baseline + 최빈값 결측치 처리 submission

public 기준 0.7027959332

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv('./drive/MyDrive/open/train.csv')
test_df = pd.read_csv('./drive/MyDrive/open/test.csv')

train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

A = pd.DataFrame(train_x.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_x.shape[0]].index)
train_x.drop(columns=a,inplace=True)
test_x.drop(columns=a, inplace=True)

mode_list=[]
for c in train_x.columns[4:]:
  mode_list.append(train_x[c].median())

for i,c in enumerate(train_x.columns[4:]):
  train_x[c].fillna(mode_list[i],inplace=True)
  test_x[c].fillna(mode_list[i],inplace=True)

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

Done.


In [ ]:
ttrain_y = train_x['Y_Class']
train_y = train_x['Y_Quality']
train_x = train_x.drop(columns=['Y_Class','Y_Quality'])

In [ ]:
clf = catboost.CatBoostRegressor(learning_rate=0.05, iterations=500, verbose=0)
clf.fit(train_x, train_y, early_stopping_rounds=100, cat_features=['PRODUCT_CODE', 'LINE'])

In [ ]:
pre_preds = clf.predict(test_x)
pre_preds

array([0.530137  , 0.53327221, 0.53086611, 0.52503777, 0.53181666,
       0.53079102, 0.5324302 , 0.52478191, 0.52410759, 0.53219965,
       0.53299099, 0.53396359, 0.5344585 , 0.52488223, 0.52557672,
       0.52612265, 0.53089421, 0.52796687, 0.530229  , 0.53210978,
       0.52792173, 0.53238021, 0.52710297, 0.52697049, 0.52740962,
       0.52739197, 0.52887602, 0.52670618, 0.52598408, 0.53136542,
       0.52651401, 0.53255908, 0.52634627, 0.52910493, 0.53137576,
       0.52436228, 0.52600856, 0.51990666, 0.55013182, 0.52578662,
       0.53041981, 0.52544719, 0.52428312, 0.52741608, 0.52916702,
       0.53012025, 0.53066789, 0.51986835, 0.52860405, 0.51976537,
       0.53117606, 0.51740275, 0.52350291, 0.52228256, 0.52391737,
       0.52323838, 0.52896477, 0.52882741, 0.52347347, 0.52334349,
       0.52224365, 0.52407559, 0.52700852, 0.5295281 , 0.52381149,
       0.52304875, 0.52294655, 0.52942043, 0.52687685, 0.52698758,
       0.52728004, 0.51687885, 0.53092859, 0.52855246, 0.53097

In [ ]:
a = train_df[['Y_Class','Y_Quality']].groupby('Y_Class').agg(['mean', 'min', 'max', 'count'])
a

Y_Quality                          
             mean       min       max count
Y_Class                                    
0        0.520837  0.500856  0.525067    88
1        0.530253  0.525086  0.534843   407
2        0.542031  0.534951  0.578841   103

In [ ]:
preds=[]
for p in pre_preds:
  if p<=a[('Y_Quality','max')][0]:
    preds.append(0)
  elif p<=a[('Y_Quality','min')][2]:
    preds.append(1)
  else:
    preds.append(2)

In [ ]:
submit = pd.read_csv('./drive/MyDrive/open/sample_submission.csv')
submit['Y_Class'] = preds
submit.to_csv('./baseline_submission2.csv', index=False)